In [3]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math
import torch.nn.functional as F

In [4]:
data = pd.read_csv('train_data_QUICK_START.csv')
sequences = data['sequence']
experiments = data['experiment_type']
#reactivity = data.iloc[:,4:210]

In [19]:
print(data)

         sequence_id                                           sequence  \
0       0000d87cab97  GGGAACGACUCGAGUAGAGUCGAAAAAGAUCGCCACGCACUUACGA...   
1       0000d87cab97  GGGAACGACUCGAGUAGAGUCGAAAAAGAUCGCCACGCACUUACGA...   
2       0001ca9d21b0  GGGAACGACUCGAGUAGAGUCGAAAAGGUGGCCGGCAGAAUCGCGA...   
3       0001ca9d21b0  GGGAACGACUCGAGUAGAGUCGAAAAGGUGGCCGGCAGAAUCGCGA...   
4       00021f968267  GGGAACGACUCGAGUAGAGUCGAAAACAUUGUUAAUGCCUAUAUUA...   
...              ...                                                ...   
335611  fffee332db3a  GGGAACGACUCGAGUAGAGUCGAAAAGAUAUGGAAGUAGAACUACC...   
335612  ffff7786573c  GGGAACGACUCGAGUAGAGUCGAAAAGAAGACGUGACGAAAAGUCA...   
335613  ffff7786573c  GGGAACGACUCGAGUAGAGUCGAAAAGAAGACGUGACGAAAAGUCA...   
335614  ffffa291ee37  GGGAACGACUCGAGUAGAGUCGAAAAGAUAUGGAAUGCGAAGAACC...   
335615  ffffa291ee37  GGGAACGACUCGAGUAGAGUCGAAAAGAUAUGGAAUGCGAAGAACC...   

       experiment_type                                       dataset_name  \
0              2A3_MaP

In [17]:
#dms, a3 = data[data.experiment_type=='DMS_MaP'].reset_index(drop=True), data[data.experiment_type=='2A3_MaP'].reset_index(drop=True)

In [62]:
'''#update NaN to csv file
m = reactivity.mean(axis=1)
for i, col in enumerate(reactivity):
    # using i allows for duplicate co lumns
    # inplace *may* not always work here, so IMO the next line is preferred
    # df.iloc[:, i].fillna(m, inplace=True)
    reactivity.iloc[:, i] = reactivity.iloc[:, i].fillna(m)
reactivity.to_csv('ReactValues',sep=',', index=False, encoding='utf-8')'''

In [22]:
reactivity = pd.read_csv('ReactValues')

In [2]:
# Split each string into characters, creating new columns
char_columns = sequences.str.split('', expand=True)

# Drop the first empty column (resulting from splitting an empty string at the beginning)
char_columns = char_columns.drop(columns=0)
# Concatenate the new columns with the original DataFrame
df = pd.concat([experiments, char_columns], axis=1)
df = df.fillna(-1)

NameError: name 'sequences' is not defined

In [68]:
class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.sequences = x_data
        self.reactivities = y_data

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        rna_sequence = torch.tensor(self.process_input(self.sequences.iloc[index, :]).values)
        reactivity_values = torch.tensor(self.reactivities.iloc[index, :].values)


        return rna_sequence, reactivity_values

    def process_input(self, rna_sequence):
        # Implement your logic to process the input RNA sequence (e.g., convert to numerical representation)
        # Return the processed input sequence
        X_train_encoded = rna_sequence.apply(self.encode_sequence)
        return X_train_encoded
    
    def encode_sequence(self, item):
        encoding = {'A': 1, 'C': 2, 'G': 3, 'U': 4, '2A3_MaP':5, 'DMS_MaP':6}
        if item == '':
             return -1
        elif item == -1:
             return -1
        if item in encoding:
            return encoding[item]
        else:
                # Handle unexpected characters (e.g., print a warning)
                print(f"Warning: Unexpected character '{item}' in sequence.")
        return np.nan

In [69]:
dataset = CustomDataset(df, reactivity)

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [9]:
#Creates a network for passing information forward through the model using two linear layers and a non-linear activation function
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        #2 linear and one nonlinear layer
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()
#overwrites the nn.Module forward function to pass the data through the model
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [6]:
#allows the model to understand the position of objects relative to a sequence
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        #creates an empty tensor
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        #uses a sin and cosine function with different frequencies to "plot the points"
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [27]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        # The purpose of nn.embedding is to take discrete tokens, like words, and change them into more vectors that can be operated on. However,
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        #gives information about the position to the model
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        #Creates the amount of encoder layers that the user specifies
        encoder_layers = TransformerEncoderLayer(d_model, num_heads, d_ff, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
# The purpose of the mask is to hide some of the parts of the sequence from each other to prevent the 
# model from gaining information on future or irrelevant parts of the sequence
    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        
        output = self.fc(dec_output)
        return output

In [ ]:
# Create the model
input_dim = 7  # A,C,G,U,DMS_MaP, 2A3_MaP, PAD
d_model = 512 
nhead = 8
num_encoder_layers = 6
num_decoder_layers = 6
dim_feedforward = 2048
max_seq_length = 208  # Adjust as per your sequence length

model = Transformer(input_dim, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, max_seq_length)

In [ ]:
def loss(pred,target):
    p = pred[target['mask'][:,:pred.shape[1]]]
    y = target['react'][target['mask']].clip(0,1)
    loss = F.l1_loss(p, y, reduction='none')
    loss = loss[~torch.isnan(loss)].mean()
    
    return loss

class MAE(Metric):
    def __init__(self): 
        self.reset()
        
    def reset(self): 
        self.x,self.y = [],[]
        
    def accumulate(self, learn):
        x = learn.pred[learn.y['mask'][:,:learn.pred.shape[1]]]
        y = learn.y['react'][learn.y['mask']].clip(0,1)
        self.x.append(x)
        self.y.append(y)

    @property
    def value(self):
        x,y = torch.cat(self.x,0),torch.cat(self.y,0)
        loss = F.l1_loss(x, y, reduction='none')
        loss = loss[~torch.isnan(loss)].mean()
        return loss